In [1]:
"""knowlede based matchmaking system
This will match profiles based on their waited compatibility scores across multiple attributes
the kbs maintains profiles and user specific rules to deter mine matches"""

'knowlede based matchmaking system\nThis will match profiles based on their waited compatibility scores across multiple attributes\nthe kbs maintains profiles and user specific rules to deter mine matches'

In [8]:
class knowledgeBase:
    """"this will store user profiles with their unique identifiers 
    Acts as the knowledge repository for the matchmaking system"""
    def __init__(self):
        self.profiles = {} #{profile_id: profile_data}
        self.next_id = 1 #auto id counter
    
    def add_profile (self, profile_data):
        """Adds a new profile to the knowledge base with automatic assignment of an ID
        param profile_data: a dictionary of profile attributes
        returns: the profile ID"""
        profile_id = self.next_id
        self.next_id += 1
        return profile_id
    
    def remove_profile(self, profile_id):
        #this removes profiles from the knowledge base
        if profile_id in self.profiles:
            del self.profiles[profile_id]
    
    def get_profile(self, profiel_id):
        #retrieves a profile by its ID
        return self.profiles.get(profiel_id)
    
    def get_all_profiles(self):
        return list(self.profiles.values())
    

class Matchmaker:
    """
    this implemets the matchmaking logic using compatibility rules and weights 
    leveraging the KB to find the optimal matches for a given profile
    """
    def __init__(self, knowledge_base, weights = None, max_age_diff = 20):
        """
        :param knowlwdge_base: a knowledge_base instance
        param weights: Dictionary conataining attribute weights fora matchmaking (sum to 1)
        param max_age_diff: Maximum age diff allowd between profiles
        """
        self.knowledge_base = knowledgeBase()
        self.weights = weights or {'age': 0.3, 'interest':0.5, 'location': 0.2}#default weights
        self.max_age_diff = max_age_diff #default max age diff

    def calculate_compatibility(self, profile_a, profile_b):
        """"1
        this calculates the compatibilty score between two profiles
        it implements domain specidic matching rules
        -Age: normalized inverse age gap
        -Interest: Jaccard similarity of interests
        -Location: Exact match check

        """
        #age compatibitlty(normalized inverse difference)
        age_diff = abs(profile_a['age'] - profile_b('age'))#absolute age difference
        age_score = max(0, 1-(age_diff/self.max_age_diff))#normalized inverse age difference

        #interest comaptibility (Jaccard similarity)
        interest_a = set(profile_a['interests'])
        interest_b = set(profile_b['interests'])
        intersection = len(interest_a & interest_b)#common interest 
        union = len(interest_a | interest_b)#all interests
        jaccard_score = (intersection / union) * self.weights['interest']#jaccard similarity

        #location comaptibitlty
        location_score = (self.weights['location'] if profile_a['location'] == profile_b['location'] else 0)
        
        return age_score + jaccard_score + location_score
    
    def find_matches(self, user_id, top_n = 5):
        """finds the top N  matches for a given user profile based on caompatiblity scores
        param user_id: the ID of the user to find matches for
        param limit: the maximum number of matches to return
        returns: a list of the best matching profiles"""
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []
        #calculate compatibility scores
        matches = {}
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] != user_id:
                matches[profile['id']] = self.calculate_compatibility(target_profile, profile)
        
        #sort the profiles by their scores
        sorted_profiles = sorted(matches.items(), key = lambda x: x[1], reverse = True)
        return sorted_profiles[:top_n]

if __name__ == '__main__':
    #initialize the knowledgebase
    kb = knowledgeBase()

    #populate with sample profiles
    profiles = [
        {'age': 25, 'interests': ['hiking', 'biking'], 'location': 'NY'},
        {'age': 30, 'interests': ['biking', 'reading'], 'location': 'NY'},
        {'age': 28, 'interests': ['biking', 'movies'], 'location': 'CA'},
        {'age': 27, 'interests': ['hiking', 'movies'], 'location': 'CA'},
        {'age': 26, 'interests': ['reading', 'movies'], 'location': 'NY'}
    ]
    for p in profiles:
        kb.add_profile(p)

    #initialize the matchmaker
    matchmaker = Matchmaker(
        knowledge_base  = kb,
        weights = {'age': 0.3, 'interest': 0.5, 'location': 0.2},
        max_age_diff = 25
    )


    #find matches for the firs profile
    print("Top matches for profile 1: ")
    for match in matchmaker.find_matches(1, top_n=3):
        print(f"profile{match[0]} with compatibilty score: {match[1]}")                

    


Top matches for profile 1: 
